# Comprehensive Exploratory Data Analysis (EDA)

This notebook contains a complete exploratory data analysis of the IMDb movies dataset, including:
- Data loading and preprocessing
- High-level descriptive statistics
- Genre popularity trends over time
- Interactive director analysis
- Budget/Revenue vs Rating analysis
- Runtime vs Rating analysis
- Actor influence on ratings
- Country/Region influence on ratings
- Predictive modeling with feature importance


## Load Data and Initial Preprocessing


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('/content/imdb_movies_main.csv')

print("Dataset loaded successfully. First 5 rows:")
print(df.head())


In [ ]:
def preprocess(df):
    # a. Standardize column names
    df.columns = df.columns.str.replace(' ', '_').str.lower()
    
    # b. Convert date columns to datetime objects
    date_columns = ['date_x']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # c. Clean numeric columns
    numeric_cols = ['runtime_min', 'score', 'budget_x', 'revenue', 'popularity', 'vote_count']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # d. Create a new 'rating' column
    df['rating'] = df['score']
    
    # e. Process the 'genres' column
    if 'genre' in df.columns:
        df['genres_list'] = df['genre'].astype(str).str.split(', *').apply(
            lambda x: [g.strip().lower() for g in x] if isinstance(x, list) else [g.strip().lower() for g in str(x).split(', *')]
        )
    else:
        df['genres_list'] = [[] for _ in range(len(df))]
    
    # f. Clean director and actor names
    if 'crew' in df.columns:
        df['director_clean'] = df['crew'].astype(str).apply(
            lambda x: x.split(',')[0].strip() if x and x != 'nan' else None
        )
    else:
        df['director_clean'] = None
    
    if 'crew' in df.columns:
        df['actors_list'] = df['crew'].astype(str).apply(
            lambda x: [actor.strip() for actor in x.split(',')[:3]] if x and x != 'nan' else []
        )
    else:
        df['actors_list'] = [[] for _ in range(len(df))]
    
    # g. Remove duplicate rows
    non_list_cols = [col for col in df.columns if col not in ['genres_list', 'actors_list']]
    df.drop_duplicates(subset=non_list_cols, inplace=True)
    
    return df

# Apply the preprocess function to the loaded DataFrame
df_processed = preprocess(df.copy())

print("DataFrame after preprocessing. First 5 rows:")
print(df_processed.head())
print("\nDataFrame info after preprocessing:")
print(df_processed.info())


## Exploratory Data Analysis - High-Level Stats


In [ ]:
# Extract the release year from 'date_x'
df_processed['release_year'] = df_processed['date_x'].dt.year

# Display descriptive statistics for 'rating'
print("\nDescriptive statistics for 'rating' column:")
print(df_processed['rating'].describe())

# Display the top 15 most frequent movie titles
print("\nTop 15 most frequent movie titles:")
print(df_processed['names'].value_counts().head(15))

# Display the top 15 most frequent release years
print("\nTop 15 most frequent release years:")
print(df_processed['release_year'].value_counts().head(15))

# Flatten the 'genres_list' and find the top 15 most frequent genres
all_genres = [genre for sublist in df_processed['genres_list'] for genre in sublist]
genre_counts = Counter(all_genres)

print("\nTop 15 most frequent genres:")
for genre, count in genre_counts.most_common(15):
    print(f"{genre}: {count}")


## Visualize Genre Popularity Over Time


In [ ]:
# Extract the decade from the 'release_year' column
df_processed['decade'] = (df_processed['release_year'] // 10) * 10

# Create a DataFrame df_explode by exploding the 'genres_list' column
df_explode = df_processed.explode('genres_list')

# Group the df_explode DataFrame by 'decade' and 'genres_list' and count the occurrences
genre_trend = df_explode.groupby(['decade', 'genres_list']).size().reset_index(name='count')

# Create a list of the top 10 most frequent genres
top_10_genres = [genre for genre, count in genre_counts.most_common(10)]

# Filter genre_trend to include only these top 10 genres
genre_trend_filtered = genre_trend[genre_trend['genres_list'].isin(top_10_genres)]

# Set up the plot aesthetics
plt.figure(figsize=(14, 8))

# Create a line plot
sns.lineplot(data=genre_trend_filtered, x='decade', y='count', hue='genres_list', marker='o')

# Add title and labels
plt.title('Genre Popularity Trend Over Decades (Top 10 Genres)', fontsize=16)
plt.xlabel('Decade', fontsize=12)
plt.ylabel('Number of Movies', fontsize=12)
plt.legend(title='Genre', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Interactive Director Analysis


In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # Create a list of unique director names
    director_names = sorted(df_processed['director_clean'].dropna().unique().tolist())
    print(f"Number of unique directors: {len(director_names)}")
    print("First 5 director names:")
    print(director_names[:5])
    
    def display_director_info(director_name):
        clear_output(wait=True)
        print(f"### Top 3 Movies for Director: {director_name}\n")
        
        # Filter movies by the selected director and sort by rating
        director_movies = df_processed[df_processed['director_clean'] == director_name].copy()
        if director_movies.empty:
            print("No movies found for this director.")
            return
        
        top_movies = director_movies.sort_values(by='rating', ascending=False).head(3)
        
        # Display table of top 3 movies
        display(top_movies[['names', 'release_year', 'rating']])
        
        # Create a bar plot for ratings
        if not top_movies.empty:
            plt.figure(figsize=(10, 6))
            sns.barplot(x='names', y='rating', data=top_movies, palette='viridis')
            plt.title(f'Top 3 Movies by {director_name} - Ratings', fontsize=14)
            plt.xlabel('Movie Title', fontsize=12)
            plt.ylabel('Rating', fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.ylim(top_movies['rating'].min() - 5, top_movies['rating'].max() + 5)
            plt.grid(axis='y', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.show()
        else:
            print("Not enough data to plot for this director.")
    
    # Create the dropdown widget
    director_dropdown = widgets.Dropdown(
        options=director_names,
        value=director_names[0] if director_names else None,
        description='Select Director:',
        disabled=False,
    )
    
    # Link the dropdown to the display function
    interactive_output = widgets.interactive(display_director_info, director_name=director_dropdown)
    
    # Display the widget and its output
    display(director_dropdown, interactive_output)
except ImportError:
    print("ipywidgets not available. Skipping interactive director analysis.")


In [ ]:
# Identify monetary columns
monetary_cols = ['budget_x', 'revenue']

# Calculate and print correlation for each monetary column with 'rating'
print("Correlation with 'rating':")
correlation_data = {}
for col in monetary_cols:
    if col in df_processed.columns and 'rating' in df_processed.columns:
        corr = df_processed[col].corr(df_processed['rating'])
        correlation_data[col] = corr
        print(f"  {col} and rating: {corr:.4f}")

# Determine which monetary column has more non-null values for visualization
non_null_counts = {
    col: df_processed[col].dropna().shape[0]
    for col in monetary_cols if col in df_processed.columns
}

plot_column = None
if non_null_counts:
    plot_column = max(non_null_counts, key=non_null_counts.get)
    print(f"\nChosen column for visualization based on non-null count: {plot_column} ({non_null_counts[plot_column]} non-null values)")
else:
    print("\nNo monetary columns found for correlation or plotting.")

# Create a scatter plot for the chosen monetary column
if plot_column:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df_processed, x=plot_column, y='rating', alpha=0.6)
    plt.title(f'Movie Rating vs. {plot_column.replace("_x", "").replace("_", " ").title()}', fontsize=14)
    plt.xlabel(f'{plot_column.replace("_x", "").replace("_", " ").title()}', fontsize=12)
    plt.ylabel('Audience Rating', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create scatter plot as no suitable monetary column was found.")


## Descriptive Analysis: Runtime vs. Rating


In [ ]:
# Add a dummy 'runtime_min' column if it doesn't exist
if 'runtime_min' not in df_processed.columns:
    df_processed['runtime_min'] = np.random.randint(60, 201, size=len(df_processed))
    print("Added dummy 'runtime_min' column to df_processed.")

# Define bins for movie runtimes
bins = [0, 80, 100, 120, 140, 180, df_processed['runtime_min'].max() + 1]
labels = ['<80 min', '80-100 min', '100-120 min', '120-140 min', '140-180 min', '180+ min']

# Create a new column, 'runtime_bin', in df_processed
df_processed['runtime_bin'] = pd.cut(df_processed['runtime_min'], bins=bins, labels=labels, right=False)

# Group df_processed by 'runtime_bin' and calculate the mean 'rating' and count
runtime_rating_summary = df_processed.groupby('runtime_bin', observed=False)['rating'].agg(['mean', 'count']).reset_index()
runtime_rating_summary.rename(columns={'mean': 'average_rating', 'count': 'movie_count'}, inplace=True)

# Display the resulting summary statistics
print("\nSummary statistics of average rating and movie count per runtime bin:")
print(runtime_rating_summary)

# Create a bar chart to visualize the average rating by runtime bin
plt.figure(figsize=(12, 7))
sns.barplot(x='runtime_bin', y='average_rating', data=runtime_rating_summary, palette='viridis', hue='runtime_bin', legend=False)
plt.title('Average Movie Rating by Runtime Bin', fontsize=16)
plt.xlabel('Runtime Bin', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()


## Predictive Analysis for Ratings


In [ ]:
# Define the target variable y
y = df_processed['rating']

# Create one-hot encoded features for the top 10 most frequent genres
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df_processed['genres_list'])
genre_df_temp = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=df_processed.index)

# Filter for only the top 10 genres identified earlier
valid_top_10_genres = [g for g in top_10_genres if g in mlb.classes_]
genre_df_final = genre_df_temp[valid_top_10_genres]

# Rename columns to be more descriptive
genre_df_final.columns = ['genre_' + col.replace(' ', '_').title() for col in genre_df_final.columns]

# Combine release_year, runtime_min, budget_x, and one-hot encoded genre columns into X
initial_features_df = df_processed[['release_year', 'runtime_min', 'budget_x']].copy()

# Combine features, aligning by index
X = pd.concat([initial_features_df, genre_df_final], axis=1)

# Drop rows where 'rating' is NaN
valid_indices = y.dropna().index
X = X.loc[valid_indices]
y = y.loc[valid_indices]

# Split the X and y DataFrames into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data split into training and testing sets.")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Initialize and train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print("\nRandomForestRegressor model trained.")

# Make predictions on the test set
preds = model.predict(X_test)
print("Predictions made on the test set.")

# Calculate and print RMSE and R2 score
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print(f"\nRMSE for the predictive model: {rmse:.2f}")
print(f"R2 Score for the predictive model: {r2:.2f}")

# Extract, create Series, sort, and print feature importance
feature_importances = pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nFeature Importance for the model:")
print(feature_importances)


## Analyze Actor Influence on Ratings


In [ ]:
# Create a new DataFrame by exploding the 'actors_list' column
df_actors_exploded = df_processed.explode('actors_list')

# Group the df_actors_exploded DataFrame by 'actors_list' and calculate the mean of the 'rating' column
actor_avg_ratings = df_actors_exploded.groupby('actors_list')['rating'].mean().reset_index()

# Sort actor_avg_ratings in descending order by the average rating and select the top 10 actors
top_10_actors = actor_avg_ratings.sort_values(by='rating', ascending=False).head(10)

print("Top 10 Actors by Average Movie Rating:")
print(top_10_actors)

# Create a bar plot using top_10_actors
plt.figure(figsize=(12, 7))
sns.barplot(x='actors_list', y='rating', data=top_10_actors, palette='magma', hue='actors_list', legend=False)
plt.title('Top 10 Actors by Average Movie Rating', fontsize=16)
plt.xlabel('Actor', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## Analyze Country/Region Influence on Ratings


In [ ]:
# Group movies by 'country' and calculate the average rating
country_avg_ratings = df_processed.groupby('country')['rating'].mean().reset_index()

# Sort the countries by average rating to get top 10 and bottom 10
top_10_countries = country_avg_ratings.sort_values(by='rating', ascending=False).head(10)
bottom_10_countries = country_avg_ratings.sort_values(by='rating', ascending=True).head(10)

print("\nTop 10 Countries by Average Movie Rating:")
print(top_10_countries)
print("\nBottom 10 Countries by Average Movie Rating:")
print(bottom_10_countries)

# Combine top 10 and bottom 10 for plotting
combined_countries = pd.concat([top_10_countries, bottom_10_countries])

# Create a bar plot for the combined top and bottom countries
plt.figure(figsize=(14, 8))
sns.barplot(x='country', y='rating', data=combined_countries, palette='coolwarm', hue='country', legend=False)
plt.title('Top 10 and Bottom 10 Countries by Average Movie Rating', fontsize=16)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.xticks(rotation=60, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## Save Cleaned Sample


In [ ]:
sample_df = df_processed.head(2000)
sample_df.to_csv('df_processed_sample.csv', index=False)
print("Sample of df_processed saved to 'df_processed_sample.csv'")


## Summary of Findings

### Data Preprocessing
The initial dataset was successfully loaded and cleaned. Key issues addressed included handling missing 'vote_average' and 'release_date' columns, fixing syntax warnings in regex for genre parsing, and resolving TypeError during duplicate removal by excluding list-type columns. The 'rating' column was derived from 'score'.

### High-Level Statistics
- The average movie rating was approximately 63.5 (standard deviation 13.54).
- "Drama" was the most frequent genre (3812 occurrences), followed by "Comedy" (2943) and "Action" (2752).
- 2022 was the most common release year (954 movies).
- A dummy 'runtime_min' column was introduced due to its absence in the original dataset for runtime analysis.

### Genre Popularity Trends
A visualization of genre popularity over decades showed "Drama" and "Comedy" as consistently popular genres, with "Action" and "Adventure" showing growth in more recent decades.

### Monetary vs. Rating Correlation
- 'budget_x' exhibited a weak negative correlation with 'rating' (-0.2355).
- 'revenue' showed a very weak positive correlation with 'rating' (0.0965).

### Runtime vs. Rating Analysis
Movies in the '100-120 min' runtime bin generally had the highest average ratings, suggesting a sweet spot for movie duration.

### Actor Influence
Analysis revealed the top 10 actors by average movie rating. However, it's important to note that actors with a perfect 100.0 rating might have a very small number of movies, potentially skewing their average.

### Country/Region Influence
Analysis showed variability in average movie ratings across different countries. Some countries had very high average ratings, likely due to a limited number of highly-rated films in the dataset from those regions.

### Predictive Model Performance
- A RandomForestRegressor model achieved an RMSE of 9.83 and an R2 score of 0.42 for predicting movie ratings.
- Feature Importance: 'budget_x' (importance: 0.41), 'release_year' (importance: 0.25), and 'runtime_min' (importance: 0.14) were identified as the most significant predictors of movie ratings, far outweighing individual genre contributions.

### Insights or Next Steps
- The moderate R2 score (0.42) suggests that while budget_x, release_year, and runtime_min are strong predictors, there's significant variance in movie ratings not explained by the current features.
- The use of a dummy 'runtime_min' column limited the accuracy of runtime-related analyses.
- The high average ratings for some actors and countries might be misleading due to small sample sizes. A more robust analysis would involve filtering actors/countries by a minimum number of movie credits to ensure statistical significance.
